In [1]:
import os
%cd /home/iir/work/ben/NCKU/IIR/SemEval2025_Task9
!pwd
import inference

/home/iir/work/ben/NCKU/IIR/SemEval2025_Task9
/home/iir/work/ben/NCKU/IIR/SemEval2025_Task9


/root/anaconda3/envs/augllms/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_data_path = 'data/incidents_val.csv'

label_hazard_category = 'models/ST1/hazard_category/hazard-category_label_encoder.npy'
label_product_category = 'models/ST1/product_category/product-category_label_encoder.npy'

weight_hazard_category = 'models/ST1/hazard_category/best_st1_hazard_category_large_v7_500.pt'
weight_product_category = 'models/ST1/product_category/best_st1_product_category_large_v7_500.pt'

bert_model_name = "microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract"
# bert_model_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract"

# Run inference
predicted_hazard_category, predicted_probs = inference.main(test_data_path, label_hazard_category, weight_hazard_category, bert_model_name, max_len=500) 

predicted_product_category, predicted_probs = inference.main(test_data_path, label_product_category, weight_product_category, bert_model_name, max_len=500)

Predicting: 100%|██████████| 565/565 [00:08<00:00, 62.81it/s]


In [3]:
import pandas as pd 
predictions = pd.DataFrame()
predictions["hazard-category"] = predicted_hazard_category
predictions["product-category"] = predicted_product_category 
# predictions

In [4]:
# save predictions to a new folder:
from shutil import make_archive
name_folder = "BERT_best_large_500_V7"

os.makedirs(f'submission/{name_folder}', exist_ok=True)
predictions.to_csv(f'submission/{name_folder}/submission.csv')

# zip the folder (zipfile can be directly uploaded to codalab):
make_archive(f'submission/{name_folder}', 'zip', f'submission/{name_folder}')

'/home/iir/work/ben/NCKU/IIR/SemEval2025_Task9/submission/BERT_best_large_500_V7.zip'